In [2]:
import pandas as pd
import re

In [3]:
positions = ['1b','2b', '3b', 'ss', 'rf', 'cf', 'lf', ' p ', ' c ', 'middle', 'right', 'left', 'center']

def quality(count):
    if len(count) > 0:
        count_list = count[0].split(' ')
        if len(count_list) == 2:
            pitches = count_list[1]
            if len(pitches) >= 7:
                return 1

def remove_dash(play):
    for w in play:
        if w[-1] == '-':
            w = w[:-1]
    return play

def field_position(play):
    for word in play:
        if word in positions:
            return word 
        
def right(position):
    if (position == '1b') or (position == '2b') or (position == 'rf') or (position == 'right'):
        return 1
    else:
        return 0
    
def center(position):
    if (position == 'p') or (position == 'c') or (position == 'cf') or (position == 'middle') or (position == 'center'):
        return 1
    else:
        return 0
    
def left(position):
    if (position == '3b') or (position == 'ss') or (position == 'lf') or (position == 'left'):
        return 1
    else:
        return 0
    
def runs(end_of_inning):
    if len(end_of_inning) > 0:
        end_list = end_of_inning[0].split(" ")
        runs = end_list[3]
        return runs
    
def hits(end_of_inning):
    if len(end_of_inning) > 0:
        end_list = end_of_inning[0].split(" ")
        hits = end_list[4][2:]
        return hits

def errors(end_of_inning):
    if len(end_of_inning) > 0:
        end_list = end_of_inning[0].split(" ")
        errors = end_list[5][2:]
        return errors
        
        

Read in the csv, clean, and make first columns for play and team. Add a column for the play in a list and find player name by finding first value in the list

In [4]:
test = pd.read_csv('test.csv')
test[test.columns[0]] = test[test.columns[0]].apply(lambda x: x.split('\n'))
test['team_name'] = test[test.columns[0]].apply(lambda x: x[0])
test['play'] = test[test.columns[0]].apply(lambda x: x[1])
test['team_name'] = test['team_name'].str.strip('title=;\r')
name_play = test.drop(columns = [test.columns[0]])
name_play['play'] = name_play['play'].apply(lambda x: x.strip('\r').replace(', ', '-'))
name_play['play_list'] = name_play['play'].apply(lambda x: x.split(' '))
name_play['name'] = name_play['play_list'].apply(lambda x: x[0])

In [5]:
name_play['play']

0      Nickles struck out swinging-out at first c to ...
1                  Perez popped up to 2b (2-2 BBFFFFFF).
2             Jordan homered to right field-RBI (1-0 B).
3                     Garcia flied out to cf (2-2 FKBB).
4                              Batting ends R: 1 H:1 E:0
                             ...                        
121               LaValley-A. pinch hit for Lyons,Grace.
122        LaValley-A. struck out swinging (3-2 BFKBBS).
123    Romero-S. singled to left field-RBI (0-2 KS); ...
124               Clifton-C. fouled out to 1b (1-2 KBK).
125                            Batting ends R: 1 H:3 E:0
Name: play, Length: 126, dtype: object

Make new columns for count, the various types of plays, and quality at bats (6 or more pitches)

In [6]:
name_play['count'] = name_play['play'].apply(lambda x: re.findall('\(.+\)', x))
name_play['pop_up'] = name_play['play'].apply(lambda x: re.findall('popped up to \w+', x))
#may need to update with strike out looking
name_play['K'] = name_play['play'].apply(lambda x: re.findall('struck out swinging', x))
name_play['home_run'] = name_play['play'].apply(lambda x: re.findall('homered to \w+ \w+', x))
name_play['fly_ball'] = name_play['play'].apply(lambda x: re.findall('flied out to \w+', x))
name_play['ground_out'] = name_play['play'].apply(lambda x: re.findall('grounded out to \w+', x))
name_play['lined_out'] = name_play['play'].apply(lambda x: re.findall('lined out to \w+', x))
name_play['scoring'] = name_play['play'].apply(lambda x: x.split(';')).apply(lambda x: x[1:])
name_play['quality_ab'] = name_play['count'].apply(quality)
name_play['play_list'] = name_play['play_list'].apply(remove_dash)

In [7]:
name_play['end of inning'] = name_play['play'].apply(lambda x: re.findall('Batting ends R: \d+ H:\d+ E:\d+', x))

Find the position in play_list (the play split on spaces) and assign to right, left, or center

In [8]:
name_play['field_position'] = name_play['play_list'].apply(field_position)
name_play['right'] = name_play['field_position'].apply(right)
name_play['left'] = name_play['field_position'].apply(left)
name_play['center'] = name_play['field_position'].apply(center)
name_play["Runs"] = name_play['end of inning'].apply(runs)
name_play["Hits"] = name_play['end of inning'].apply(hits)
name_play["Errors"] = name_play['end of inning'].apply(errors)
name_play[:40]

,team_name,play,play_list,name,count,pop_up,K,home_run,fly_ball,ground_out,...,scoring,quality_ab,end of inning,field_position,right,left,center,Runs,Hits,Errors
0,UCLA,Nickles struck out swinging-out at first c to ...,"[Nickles, struck, out, swinging-out, at, first...",Nickles,[(0-2 KKFS)],[],[struck out swinging],[],[],[],...,[],NaN,[],1b,1,0,0,None,None,None
1,UCLA,Perez popped up to 2b (2-2 BBFFFFFF).,"[Perez, popped, up, to, 2b, (2-2, BBFFFFFF).]",Perez,[(2-2 BBFFFFFF)],[popped up to 2b],[],[],[],[],...,[],1.0,[],2b,1,0,0,None,None,None
2,UCLA,Jordan homered to right field-RBI (1-0 B).,"[Jordan, homered, to, right, field-RBI, (1-0, ...",Jordan,[(1-0 B)],[],[],[homered to right field],[],[],...,[],NaN,[],right,1,0,0,None,None,None
3,UCLA,Garcia flied out to cf (2-2 FKBB).,"[Garcia, flied, out, to, cf, (2-2, FKBB).]",Garcia,[(2-2 FKBB)],[],[],[],[flied out to cf],[],...,[],NaN,[],cf,0,0,1,None,None,None
4,UCLA,Batting ends R: 1 H:1 E:0,"[Batting, ends, R:, 1, H:1, E:0]",Batting,[],[],[],[],[],[],...,[],NaN,[Batting ends R: 1 H:1 E:0],None,0,0,0,1,1,0
5,Oklahoma,Batting starts,"[Batting, starts]",Batting,[],[],[],[],[],[],...,[],NaN,[],None,0,0,0,None,None,None
6,Oklahoma,Romero-S. grounded out to 2b (1-2 FBF).,"[Romero-S., grounded, out, to, 2b, (1-2, FBF).]",Romero-S.,[(1-2 FBF)],[],[],[],[],[grounded out to 2b],...,[],NaN,[],2b,1,0,0,None,None,None
7,Oklahoma,Clifton-C. popped up to 2b (1-2 KFB).,"[Clifton-C., popped, up, to, 2b, (1-2, KFB).]",Clifton-C.,[(1-2 KFB)],[popped up to 2b],[],[],[],[],...,[],NaN,[],2b,1,0,0,None,None,None
8,Oklahoma,Alo-Jocelyn lined out to 3b (1-1 BK).,"[Alo-Jocelyn, lined, out, to, 3b, (1-1, BK).]",Alo-Jocelyn,[(1-1 BK)],[],[],[],[],[],...,[],NaN,[],3b,0,1,0,None,None,None
9,Oklahoma,Batting ends R: 0 H:0 E:0,"[Batting, ends, R:, 0, H:0, E:0]",Batting,[],[],[],[],[],[],...,[],NaN,[Batting ends R: 0 H:0 E:0],None,0,0,0,0,0,0


In [240]:
for name in name_play['play_list']:
    print(name)

['Nickles', 'struck', 'out', 'swinging-out', 'at', 'first', 'c', 'to', '1b', '(0-2', 'KKFS).']
['Perez', 'popped', 'up', 'to', '2b', '(2-2', 'BBFFFFFF).']
['Jordan', 'homered', 'to', 'right', 'field-RBI', '(1-0', 'B).']
['Garcia', 'flied', 'out', 'to', 'cf', '(2-2', 'FKBB).']
['Batting', 'ends', 'R:', '1', 'H:1', 'E:0']
['Batting', 'starts']
['Romero-S.', 'grounded', 'out', 'to', '2b', '(1-2', 'FBF).']
['Clifton-C.', 'popped', 'up', 'to', '2b', '(1-2', 'KFB).']
['Alo-Jocelyn', 'lined', 'out', 'to', '3b', '(1-1', 'BK).']
['Batting', 'ends', 'R:', '0', 'H:0', 'E:0']
['Batting', 'starts']
['Pack', 'grounded', 'out', 'to', 'p', '(0-2', 'KFF).']
['Sullivan', 'struck', 'out', 'swinging', '(1-2', 'BSKS).']
['Tautalafua', 'grounded', 'out', 'to', '2b', '(2-2', 'KBBK).']
['Batting', 'ends', 'R:', '0', 'H:0', 'E:0']
['Batting', 'starts']
['Green,Grace', 'flied', 'out', 'to', 'lf', '(0-1', 'K).']
['Knighten-S.', 'homered', 'to', 'right', 'center-RBI', '(3-0', 'BBB).']
['Mendes-N.', 'grounded', 'o

In [241]:
re.findall('\(.+\)', string)

['(3-2 BFKBBS)']